In [1]:
%matplotlib qt
from scipy.ndimage.measurements import label, find_objects
from scipy.interpolate import LinearNDInterpolator, CloughTocher2DInterpolator
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cd "C:\Users\jdavies\Google Drive\MAGPIE\data\2015\s1029_15 Reconnection C 32 Wire TS in layer 280 ns"

C:\Users\jdavies\Google Drive\MAGPIE\data\2015\s1029_15 Reconnection C 32 Wire TS in layer 280 ns


In [124]:
t=plt.imread('s1029_15 532 plasma.png')
t=t.sum(axis=2)
mask_colour=np.unique(t)[1]#find the shade of gray
tm=np.ma.masked_equal(t, mask_colour) #get the mask
tt=(-tm+4)/3
th=np.clip(tt,a_min=0, a_max=2)

In [125]:
structure=np.ones((3,3))
labeled_array, num_fringes = label(th, structure=structure)
slices=find_objects(labeled_array)
labeled_array[labeled_array==0]=-9999
labeled_array=labeled_array-1
#la=np.ma.masked_equal(labeled_array, 0)

In [276]:
class FringeNumberer:
    def __init__(self, im, slices):
        self.num_fringes=len(slices)
        self.fringe_dict={key: None for (key, value) in zip(range(num_fringes),range(num_fringes))}
        self.line,=im.axes.plot([0],[0],lw=1)
        self.im = im
        self.fringe_indexes= im.get_array().copy()
        self.fringe_labels=np.zeros_like(self.fringe_indexes)-10000
        self.initialise_fringe_map()
        self.xs = []
        self.ys = []
        self.in_a_line=False
        self.cid = im.figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        ax=self.im.axes
        print('click', event)
        if event.inaxes!=ax: 
            print('return')
            return
        if event.dblclick is True and self.in_a_line is False:
            print('Begin Line')
            ax.set_title('Dbl click to start, click to add, dbl click to end')
            self.in_a_line=True
            self.xs=[event.xdata]
            self.ys=[event.ydata]
            return
        if event.dblclick is True and self.in_a_line is True:
            print('End Line')
            self.in_a_line=False
            self.xs.append(event.xdata)
            self.ys.append(event.ydata)
            self.line.set_data(self.xs, self.ys)
            ax.set_xlim([0,la.shape[0]])
            ax.set_xlim([0,la.shape[1]])
            self.get_labels()
            self.check_labels()
            self.label_fringes()
            self.update_fringe_map()
            self.line.figure.canvas.draw()
            return
        if event.dblclick is False and self.in_a_line is True:
            print('Add segment')
            self.xs.append(event.xdata)
            self.ys.append(event.ydata)
            self.line.set_data(self.xs, self.ys)
            ax.set_xlim([0,la.shape[0]])
            ax.set_xlim([0,la.shape[1]])
            self.line.figure.canvas.draw()
            return
        
    def get_labels(self):
        d=self.fringe_indexes
        self.fringe_indexes_on_line=[]
        for i in range(len(self.xs)-1):
            x0, y0 = self.xs[i], self.ys[i] # These are in _pixel_ coordinates!!
            x1, y1 = self.xs[i+1], self.ys[i+1]
            length = int(np.hypot(x1-x0, y1-y0))
            x, y = np.linspace(x0, x1, length), np.linspace(y0, y1, length)
            # Extract the values along the line
            zi = d[y.astype(np.int), x.astype(np.int)]
            zi=[z for z in zi if z!=-10000]
            self.fringe_indexes_on_line.extend(zi)
            
    def check_labels(self):        
        fn=self.fringe_indexes_on_line
        #remove adjacent duplicates
        self.fringe_indexes_on_line=[n for i, n in enumerate(fn) if i==0 or n != fn[i-1]]
        #check there are no other duplicates
        if len(set(self.fringe_indexes_on_line)) is not len(self.fringe_indexes_on_line):
            self.im.axes.set_title('Line recrosses fringes, try again')
            self.im.axes.title.set_color('r')
            self.line.figure.canvas.draw()
            #self.fringe_indexes_on_line=[]
            return
        else:
            return
        
    def label_fringes(self):
        mode=+1
        start_index=self.fringe_dict[self.fringe_indexes_on_line[0]]
        if start_index is None:
            start_index=0
        for ind, f in enumerate(self.fringe_indexes_on_line):
            self.fringe_dict[f]=start_index+ind*mode
            
    def update_fringe_map(self):
        for f in range(self.num_fringes):
            s=slices[f]
            si=self.fringe_indexes[s]
            sl=self.fringe_labels[s]
            if self.fringe_dict[f] is not None:
                sl[si==f]=self.fringe_dict[f]
            else:
                sl[si==f]=1
        self.im.set_data(self.fringe_labels)
        
        mi=min(x for x in self.fringe_dict.values() if x is not None)
        ma=max(x for x in self.fringe_dict.values() if x is not None)
        self.im.set_clim([mi, ma])
        self.im.figure.canvas.draw()
    
    def initialise_fringe_map(self):
        for f in range(self.num_fringes):
            s=slices[f]
            si=self.fringe_indexes[s]
            sl=self.fringe_labels[s]
            sl[si==f]=1
        self.im.set_data(self.fringe_labels)
        self.im.set_clim([0, 1])
        self.im.figure.canvas.draw()
        
    def output_points_values(self):
        points=np.zeros((0,2))
        values=np.zeros((0))
        mi=min(x for x in self.fringe_dict.values() if x is not None)
        ma=max(x for x in self.fringe_dict.values() if x is not None)
        for fl in range(mi, ma+1):
            p=np.transpose(np.where(self.fringe_labelled_only==fl))
            points=np.append(points,p,axis=0)
            v=np.zeros(p.shape[0])+fl
            values=np.append(values, v)
        self.labeled_points=points
        self.labeled_values=values
        return points, values
        
    def remove_unlabelled(self):
        arr=np.zeros_like(self.fringe_labels)-10000
        for f in range(self.num_fringes):
            s=slices[f]
            si=self.fringe_indexes[s]
            sl=arr[s]
            if self.fringe_dict[f] is not None:
                sl[si==f]=self.fringe_dict[f]
        self.fringe_labelled_only=arr        
            
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('Dbl click to start, click to add, dbl click to end')

cmap = plt.cm.jet
cmap.set_under('w')
im = ax.imshow(labeled_array, cmap=cmap, interpolation='nearest')

FN = FringeNumberer(im, slices)

plt.show()

click MPL MouseEvent: xy=(199,227) xydata=(439.516129032,1645.16129032) button=1 dblclick=False inaxes=Axes(0.125,0.154114;0.775x0.691771)
click MPL MouseEvent: xy=(199,227) xydata=(439.516129032,1645.16129032) button=1 dblclick=True inaxes=Axes(0.125,0.154114;0.775x0.691771)
Begin Line
click MPL MouseEvent: xy=(522,231) xydata=(4346.77419355,1596.77419355) button=1 dblclick=False inaxes=Axes(0.125,0.154114;0.775x0.691771)
Add segment
click MPL MouseEvent: xy=(522,231) xydata=(4405.28870293,1583.68200837) button=1 dblclick=True inaxes=Axes(0.157585,0.1;0.70983x0.8)
End Line


In [277]:
FN.remove_unlabelled()
points, values=FN.output_points_values()

In [294]:
%time
li=LinearNDInterpolator(points, values)
d=li(grid_x.ravel(), grid_y.ravel())

Wall time: 0 ns


In [295]:
%time
CT=CloughTocher2DInterpolator(points, values, tol=1e-8, maxiter=1000)
d=CT(grid_x.ravel(), grid_y.ravel())

Wall time: 0 ns


In [279]:
grid_x, grid_y = np.mgrid[0:labeled_array.shape[0], 0:labeled_array.shape[1]]

In [280]:
d=CT(grid_x.ravel(), grid_y.ravel())

In [281]:
dr=d.reshape(grid_x.shape[0],grid_x.shape[1])

fig = plt.figure()
ax = fig.add_subplot(111)
im = ax.imshow(np.nan_to_num(dr), clim=[-100,100],interpolation='nearest')

In [282]:
fl=FN.fringe_labelled_only

In [249]:
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = plt.cm.jet
cmap.set_under('w')

im = ax.imshow(fl[1200:2200,1500:3500], cmap=cmap,clim=[0,fl.max()],interpolation='nearest')

In [262]:
from scipy.spatial import Delaunay

In [284]:
tri=Delaunay(points)
fig = plt.figure()
ax = fig.add_subplot(111)

ax.triplot(points[:,1], points[:,0], tri.simplices.copy())
ax.plot(points[:,1], points[:,0], 'o')
ax.invert_yaxis()
im = ax.imshow(np.nan_to_num(dr), clim=[-100,100],interpolation='nearest')


In [283]:
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = plt.cm.jet
cmap.set_under('w')

#ax.triplot(points[:,1], points[:,0], tri.simplices.copy())
ax.plot(points[:,1], points[:,0], 'o')
ax.invert_yaxis()

im = ax.imshow(fl, cmap=cmap,clim=[0,fl.max()],interpolation='nearest')

In [286]:
drm=np.ma.asarray(dr)
drm.mask=tm.mask

In [291]:
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = plt.cm.jet
cmap.set_under('w')

im = ax.imshow(drm, cmap=cmap,interpolation='nearest')

In [289]:
drm.max()

nan